## data preparation

In [ ]:
import tensorflow as tf
print(tf.config.list_physical_devices())

import gc
class FreeMemory(tf.keras.callbacks.Callback):
    def __init__(self, log_freq=None):
        super().__init__()
        self.log_freq = log_freq
    def on_epoch_end(self, epoch, logs=None):
        if self.log_freq and epoch % self.log_freq == 0:
            print(f'epoch {epoch} ended, info: {logs}')
        tf.keras.backend.clear_session()
        gc.collect()

In [ ]:
POINTS_NUM = 10
CLASSES_NUM = 2

In [ ]:
import numpy as np
points = np.load(f'/host/dissertation/proccessed_data/points_{POINTS_NUM}.npy')
q = np.load('/host/dissertation/proccessed_data/q.npy')

In [ ]:
q[q == -1] = 0
q_labels = tf.one_hot(q, CLASSES_NUM)

## plotting

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def make_dataframe(fit_history):
    df = pd.DataFrame(fit_history.history)
    df.insert(0, 'epoch', fit_history.epoch)
    return df

def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w

def plot_loss_history(fit_history, metric = 'loss', title = "", threshold_multiplier = 10):

    if isinstance(fit_history, pd.core.frame.DataFrame):
        df = fit_history
    else:
        df = make_dataframe(fit_history)

    train_1st_percentile = np.percentile(df[metric].values, 1)
    val_1st_percentile = np.percentile(df[f'val_{metric}'].values, 1)
    print(f'1st percentile of train {metric}:       {train_1st_percentile:.4e}')
    print(f'1st percentile of validation {metric}:  {val_1st_percentile:.4e}')
    
    threshold = np.maximum(train_1st_percentile, val_1st_percentile)
    fig, axes = plt.subplots(3, sharex=True, figsize=(8,6))
    axes[0].set_title(f"{title} {metric} history")
    for axis in axes:
        axis.set_ylim((0, threshold_multiplier*threshold))
        axis.plot(df['epoch'], df[metric], label='training set')
        axis.plot(df['epoch'], df[f'val_{metric}'], linestyle='dashed', label='validation set')
        threshold_multiplier /= 2
    axes[-1].legend(loc="lower left")
    axes[-1].set_xlabel("epoch no.")
    fig.supylabel(f"{metric} value")

def plot_accuracy_history(fit_history, moving_average_window = (10,), metric = 'accuracy'):
    
    if isinstance(fit_history, pd.core.frame.DataFrame):
        df = fit_history
    else:
        df = make_dataframe(fit_history)
    
    plt.figure()
    
    plt.plot(df['epoch'], df[metric], label=f'training set {metric}')
    plt.plot(df['epoch'], df[f'val_{metric}'], linestyle='dotted', linewidth=0.5, label=f'validation set {metric}')
    for w in moving_average_window:
        plt.plot(df['epoch'][w-1:], moving_average(df[f'val_{metric}'], w), linestyle='dashed', label=f'validation set {metric} moving average, w={w}')
    
    plt.xlabel('epoch no.')
    plt.legend()
    plt.show()
    

## categorical_crossentropy

In [ ]:
import cvnn.layers as complex_layers

model = tf.keras.models.Sequential()
model.add(complex_layers.ComplexInput(input_shape=(POINTS_NUM,)))
model.add(complex_layers.ComplexDense(units=64, activation='cart_relu'))
model.add(complex_layers.ComplexDense(units=CLASSES_NUM, activation='sigmoid_real'))
model.compile(optimizer=tf.keras.optimizers.SGD(clipnorm=1.), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(points, q_labels, epochs=250, validation_split=0.2, verbose=0, callbacks=[FreeMemory(10)])

In [ ]:
# plot_loss_history(history)
plot_accuracy_history(history, (40,))

In [ ]:
make_dataframe(history).to_csv('/host/dissertation/trained_models/history.csv', index=False)
df = pd.read_csv('/host/dissertation/trained_models/history.csv')

tail_size = 20
val_accuracy_tail = df['val_accuracy'][-tail_size:]
accuracy_tail = df['accuracy'][-tail_size:]
print('-------------')
print('val_accuracy')
print(f'   avg: {np.average(val_accuracy_tail):.6f}')
print(f'   med: {np.median(val_accuracy_tail):.6f}')
print(f'   std: {np.std(val_accuracy_tail):.6f}')
print('accuracy')
print(f'   max: {np.max(accuracy_tail):.6f}')
print(f'   min: {np.min(accuracy_tail):.6f}')
print('-------------\n')


### space search

In [ ]:
TAIL_SIZE = 20
EPOCHS_NUM = 30 # 250
hiddenlayers_number_space = [1,2,3]
layer_neurons_number_space = [32,64,96]
layer_activation_func_space = ['pol_tanh','pol_sigmoid','pol_selu','cart_sigmoid','cart_relu','cart_tanh'] #['cart_relu']#

import itertools
hp_list = list(itertools.product(hiddenlayers_number_space,layer_neurons_number_space,layer_activation_func_space))
param = {'hidden_layers': 0, 'neurons_num': 1, 'activation_func': 2}

In [ ]:
import os 
import time

TIMESTAMP = int(time.time())
WORKING_DIRECTORY = f'/host/dissertation/trained_models/{TIMESTAMP}/'
os.mkdir(WORKING_DIRECTORY)

import csv
HP_FILEPATH = WORKING_DIRECTORY + 'q_hp.csv'
with open(HP_FILEPATH, 'w', newline='') as f:
    hp_file_columns = ['hp_config','time','val_acc-avg','val_acc-med','val_acc-std','train_acc-max','train_acc-min']
    csv.writer(f).writerow(hp_file_columns)

In [ ]:
def hp_string(config):
    return str(config).translate(str.maketrans(',','-'," '()"))
def get_history_filename(config):
    return hp_string(config) + '-history.csv'

get_history_filename((1, 32, 'cart_relu'))

In [ ]:
def append_hp_file(hp_config, df):
    val_accuracy_tail = df['val_accuracy'][-TAIL_SIZE:]
    accuracy_tail = df['accuracy'][-TAIL_SIZE:]
    
    with open(HP_FILEPATH, 'a', newline='') as f:
        csv.writer(f).writerow([hp_string(hp_config),time.strftime("%H:%M:%S", time.localtime()),np.average(val_accuracy_tail),np.median(val_accuracy_tail),np.std(val_accuracy_tail),np.max(accuracy_tail),np.min(accuracy_tail)])

In [ ]:
import cvnn.layers as complex_layers

for hp in hp_list:
    print('START', hp)
    model = tf.keras.models.Sequential()
    model.add(complex_layers.ComplexInput(input_shape=(POINTS_NUM,)))
    for layer_no in range(hp[param['hidden_layers']]):
        model.add(complex_layers.ComplexDense(units=hp[param['neurons_num']], activation=hp[param['activation_func']]))
    model.add(complex_layers.ComplexDense(units=CLASSES_NUM, activation='sigmoid_real'))
    model.compile(optimizer=tf.keras.optimizers.SGD(clipnorm=1.), loss='binary_crossentropy', metrics=['accuracy'])
    print('      model compiled')
    print('      training...')
    history = model.fit(points, q_labels, epochs=EPOCHS_NUM, validation_split=0.2, verbose=0, callbacks=[FreeMemory()])
    print('      model trained')
    df = make_dataframe(history)
    append_hp_file(hp, df)
    print('      hp_file appended')
    df.to_csv(WORKING_DIRECTORY + get_history_filename(hp), index=False)
    print('      history saved')
    print('DONE ', hp)

## sparse_categorical_crossentropy

In [ ]:
# import cvnn.layers as complex_layers

# model = tf.keras.models.Sequential()
# model.add(complex_layers.ComplexInput(input_shape=(POINTS_NUM,)))
# model.add(complex_layers.ComplexDense(units=64, activation='cart_relu'))
# model.add(complex_layers.ComplexDense(units=CLASSES_NUM, activation='softmax_of_softmax_real_with_avg'))
# model.compile(optimizer=tf.keras.optimizers.SGD(clipnorm=1.), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# model.summary()

In [ ]:
# history = model.fit(points, q, epochs=250, validation_split=0.2, verbose=0, callbacks=[FreeMemory(10)])